In [34]:
from nustar_pysolar import planning, io
import astropy.units as u
from astropy.coordinates import SkyCoord
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [25]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2021_003.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [26]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [27]:
tstart = '2021-01-08T10:00:00'
tend = '2021-01-08T17:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

In [28]:
orbits

[[<Time object: scale='utc' format='isot' value=2021-01-08T10:11:50.000>,
  <Time object: scale='utc' format='isot' value=2021-01-08T11:14:50.000>],
 [<Time object: scale='utc' format='isot' value=2021-01-08T11:48:30.000>,
  <Time object: scale='utc' format='isot' value=2021-01-08T12:51:30.000>],
 [<Time object: scale='utc' format='isot' value=2021-01-08T13:25:10.000>,
  <Time object: scale='utc' format='isot' value=2021-01-08T14:28:10.000>],
 [<Time object: scale='utc' format='isot' value=2021-01-08T15:01:50.000>,
  <Time object: scale='utc' format='isot' value=2021-01-08T16:04:50.000>]]

In [29]:
# Get the solar parameter
from sunpy.coordinates import sun

angular_size = sun.angular_radius(t='now')
dx = angular_size.arcsec
print(dx)

975.5517028497218


In [30]:
pa = planning.get_nustar_roll(tstart, 0)
print(tstart)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

2021-01-08T10:00:00
NuSTAR Roll angle for Det0 in NE quadrant: 358.371623346487 deg


In [48]:
# Orbit 1 (West limb)
offset = [700., -300.]*u.arcsec

ind = 1
orbit = orbits[0]
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print("Orbit: {}".format(ind))
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")


# Orbit 2 (East Limb)
offset = [-800., 150.]*u.arcsec
for oi in [1, 2, 3]:
    ind = oi+1

    orbit = orbits[oi]
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
    print("Orbit: {}".format(ind))
    print(f"Solar offset: {offset}")
    print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
    print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
    print("")




Orbit: 1
Solar offset: [ 700. -300.] arcsec
Orbit start: 2021-01-08 10:11:50.000 Orbit end: 2021-01-08 11:14:50.000
Aim time: 2021-01-08 10:43:20.000 RA (deg): 289.3855 deg Dec (deg): -22.3051 deg

Orbit: 2
Solar offset: [-800.  150.] arcsec
Orbit start: 2021-01-08 11:48:30.000 Orbit end: 2021-01-08 12:51:30.000
Aim time: 2021-01-08 12:20:00.000 RA (deg): 289.9047 deg Dec (deg): -22.1589 deg

Orbit: 3
Solar offset: [-800.  150.] arcsec
Orbit start: 2021-01-08 13:25:10.000 Orbit end: 2021-01-08 14:28:10.000
Aim time: 2021-01-08 13:56:40.000 RA (deg): 289.9779 deg Dec (deg): -22.1496 deg

Orbit: 4
Solar offset: [-800.  150.] arcsec
Orbit start: 2021-01-08 15:01:50.000 Orbit end: 2021-01-08 16:04:50.000
Aim time: 2021-01-08 15:33:20.000 RA (deg): 290.0511 deg Dec (deg): -22.1403 deg



In [47]:
test1 = SkyCoord(289.3792274160115,  -22.304595055979675, unit = 'deg')
orb1 = SkyCoord(289.3855, -22.3051, unit = 'deg')
orb1.separation(test1)

<Angle 0.00582518 deg>

<Angle 0.00582518 deg>

In [38]:
import sunpy

In [39]:
sunpy.__version__

'2.0.1'

In [49]:
test1 = SkyCoord(289.898451566591, -22.158432904027155 , unit = 'deg')
orb1 = SkyCoord(289.9047,  -22.1589, unit = 'deg')
orb1.separation(test1)

<Angle 0.00580576 deg>